In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import SVHN

# Define transformation for data preprocessing
transform = transforms.Compose([
    transforms.Grayscale(),             # Convert images to grayscale
    transforms.Resize((32, 32)),        # Resize images to 32x32
    transforms.ToTensor(),              # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,)) # Normalize images
])

# Load SVHN dataset
train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Instantiate the model
model = LeNet5()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

# Evaluating the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Epoch [1/10], Step [100/1145], Loss: 2.2390
Epoch [1/10], Step [200/1145], Loss: 2.0429
Epoch [1/10], Step [300/1145], Loss: 1.4398
Epoch [1/10], Step [400/1145], Loss: 1.0704
Epoch [1/10], Step [500/1145], Loss: 0.8886
Epoch [1/10], Step [600/1145], Loss: 0.8154
Epoch [1/10], Step [700/1145], Loss: 0.7212
Epoch [1/10], Step [800/1145], Loss: 0.7132
Epoch [1/10], Step [900/1145], Loss: 0.6670
Epoch [1/10], Step [1000/1145], Loss: 0.6632
Epoch [1/10], Step [1100/1145], Loss: 0.6399
Epoch [2/10], Step [100/1145], Loss: 0.5727
Epoch [2/10], Step [200/1145], Loss: 0.5770
Epoch [2/10], Step [300/1145], Loss: 0.5670
Epoch [2/10], Step [400/1145], Loss: 0.5576
Epoch [2/10], Step [500/1145], Loss: 0.5138
Epoch [2/10], Step [600/1145], Loss: 0.5313
Epoch [2/10], Step [700/1145], Loss: 0.5238
Epoch [2/10], Step [800/1145], Loss: 0.5125
Epoch [2/10], Step [900/1145], Loss: 0.4933
E

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import SVHN
from torchvision.models import resnet18, resnet50, resnet101

# Define transformation for data preprocessing

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert images to RGB format
    transforms.Resize((32, 32)),                  # Resize images to 32x32
    transforms.ToTensor(),                        # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))         # Normalize images
])

# Load SVHN dataset
train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ResNet model
class MyResNet(nn.Module):
    def __init__(self, resnet_type='resnet18', num_classes=10):
        super(MyResNet, self).__init__()
        if resnet_type == 'resnet18':
            self.resnet = resnet18(pretrained=False)
        elif resnet_type == 'resnet50':
            self.resnet = resnet50(pretrained=False)
        elif resnet_type == 'resnet101':
            self.resnet = resnet101(pretrained=False)
        else:
            raise ValueError("Invalid ResNet type. Choose between 'resnet18', 'resnet50', and 'resnet101'.")
        
        self.resnet.fc = nn.Linear(in_features=self.resnet.fc.in_features, out_features=num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

    def train_and_evaluate(self, train_loader, test_loader, num_epochs=10):
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=0.001)

        # Training the model
        for epoch in range(num_epochs):
            self.train()
            running_loss = 0.0
            for i, (images, labels) in enumerate(train_loader):
                optimizer.zero_grad()
                outputs = self(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if (i+1) % 100 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
                    running_loss = 0.0

        # Evaluating the model
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

# User input for selecting the ResNet model
print("Select the ResNet model:")
print("1. ResNet18")
print("2. ResNet50")
print("3. ResNet101")
resnet_choice = int(input("Enter your choice (1/2/3): "))

# Model selection based on user input
if resnet_choice == 1:
    model = MyResNet(resnet_type='resnet18')
elif resnet_choice == 2:
    model = MyResNet(resnet_type='resnet50')
elif resnet_choice == 3:
    model = MyResNet(resnet_type='resnet101')
else:
    print("Invalid choice.")
    exit()

# Train and evaluate the selected model
model.train_and_evaluate(train_loader, test_loader)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Select the ResNet model:
1. ResNet18
2. ResNet50
3. ResNet101


Enter your choice (1/2/3):  1


Epoch [1/10], Step [100/1145], Loss: 1.7413
Epoch [1/10], Step [200/1145], Loss: 0.8974
